In [1]:
import pandas as pd
from pyhanlp import *
import jieba
import numpy as np
from gensim.models import word2vec

df = pd.read_excel('Lawsnote法律黑客松＿判決摘要.xlsx',sheet_name='慰撫金')

# 濾除雜質
for idx, row in df.iterrows():
    for col in df.columns:
        if type(df.iloc[idx][col]) == str:
            value = df.iloc[idx][col]
            value = value.strip()
            value = value.replace('未滿','').replace('不明','').replace('未主張','')
            value = value.replace('目前沒有工作','').replace('目前在監服刑','').replace('在監執行','')
            value = value.replace('無','').replace('目前在監服刑','').replace('在監執行','')
            df.iloc[idx][col] = value 
            
        if col == '年齡':
            if ('nan' in value) or (value == ''): df.iloc[idx][col] = 0
                    
                
                                        
# 留下必要row           
col = [c for c in df.columns if c not in ['法院']]
df = df.drop([0,1])
df = df.reset_index(drop=True)
df = df.dropna(subset=['最後判賠金額'])
df = df[df["犯罪類型"] != 'X']

In [2]:
df

,法院,主文,犯罪類型,職業,產業,階級,學歷,年齡,等級,數額,財產總額,財產種類,原告主張,原告年紀,被告主張,有利原告,有利被告,原告要求金額,最後判賠金額
0,臺灣新北地方法院107年訴字第1661號民事判決,臺灣新北地方法院民事判決107年度訴字第1661號原告0000-000000（真實姓名年籍均...,強制性交,"餐飲及溫泉會館,老闆",住宿及餐飲業,老闆,NaN,NaN,經濟狀況良好,1000000,NaN,NaN,服用藥物昏沈不能抗拒,52,,趁原告服用安眠藥法抗拒\n事後悔意從未道歉\n指稱原告設局引誘被告,NaN,800000,500000
2,臺灣高等法院107年上易字第56號民事判決,臺灣高等法院民事判決107年度上易字第56號上訴人王凱民訴訟代理人王俊傑律師被上訴人A女(0...,強制性交未遂,待業,NaN,NaN,大學肄業,NaN,NaN,52540,NaN,NaN,因酒醉不知不能抗拒,NaN,當日因飲酒過量精神恍惚\n證人之證詞互有矛盾\nDNA未為23種型別比對\n請求之賠償金額過高,被告利用原告酒醉不知不能抗拒,NaN,1300000,900000
3,臺灣新北地方法院107年訴字第727號民事判決,臺灣新北地方法院民事判決107年度訴字第727號原告林明子訴訟代理人蔣宗翰律師張君維被告徐松...,性騷擾,待業,NaN,NaN,國小畢業,NaN,NaN,36000,NaN,"投資:1,不動產:1",被告不滿原告以徒手拍打其臀部，而徒手反推原告\n原告當場倒地，造成原告背部外傷及骨折,NaN,正當防衛，傷害故意\n被告才是受害人,NaN,NaN,700000,50000
4,臺灣臺北地方法院106年醫字第16號民事判決,臺灣臺北地方法院民事判決106年度醫字第16號原告A女（真實姓名、住居所詳卷）訴訟代理人周珊...,性騷擾,醫師,醫療保健業,員工,NaN,NaN,NaN,1914876,NaN,NaN,被告趁原告尋求醫療協助之際，要求發生性行為\n醫療行為不當，致原告腿部神經病變,NaN,術前已盡評估之事\n原告為護士，於術前應已了解風險,趁病人不適且法逃脫醫師治療之依賴\n違反道德,NaN,300000,250000
5,臺灣高等法院臺中分院107年訴易字第78號民事判決,臺灣高等法院臺中分院民事判決107年度訴易字第78號原告A女（代號0000-000000）（...,強制性交,待業,NaN,NaN,專科肄業,NaN,輕度精神障礙\n存款\n領有重大傷病證明卡,0,NaN,NaN,被告明知其為低智商而強制性交,NaN,二人互有情愫\n並未暴力脅迫原告\n經濟能力不佳，不應請求高額求償金,被告明知其為低智商而強制性交,NaN,1000000,400000
6,臺灣臺中地方法院106年訴字第2586號民事判決,臺灣臺中地方法院民事判決106年度訴字第2586號原告A訴訟代理人錢炳村律師複代理人錢妍坊被...,強制性交,待業,NaN,NaN,高中畢業,NaN,NaN,0,NaN,汽車:1,因酒醉不知不能抗拒\n2次乘機性交,NaN,證詞前後矛盾\n法排除合意性交之可能,違反原告之意願2次乘機性交,NaN,800000,300000
9,臺灣嘉義地方法院107年訴字第300號民事判決,臺灣嘉義地方法院民事判決107年度訴字第300號原告0000-000000（真實姓名及住所均...,強制性交未遂,待業,NaN,NaN,NaN,NaN,,0,NaN,NaN,被告爬進原告房間\n趁其昏睡強制性交\n匿名恐嚇原告不得洩漏,20,被告沒有那麼多錢\n希望賠償金少一點,原告年紀輕，尚在追逐夢想年紀,NaN,3000000,850000
10,臺灣桃園地方法院107年訴字第561號民事判決,臺灣桃園地方法院民事判決107年度訴字第561號原告吳○○（姓名、住所詳卷）兼法定代理人吳○...,猥褻,待業,NaN,NaN,NaN,12,NaN,1048450,2540000,NaN,A女僅小六，長期被被告猥褻，致患PTSD\n期間並出現腦中有聲音、睡眠障礙、經常作惡夢、產生...,12,僅有2次猥褻行為,A女於事發時為12歲之少年，身心發育尚未成熟,NaN,400000,150000
11,臺灣基隆地方法院107年訴字第255號民事判決,臺灣基隆地方法院民事判決107年度訴字第255號原告林女（代號0000-000000，姓名年...,強制性交,待業,NaN,NaN,NaN,20,NaN,35000,NaN,NaN,趁其進房間詢問WIFI密碼之際強制性交\n造作原告林女之父、林女之母於親子關係中產生悲憤、沮...,13,請求駁回\n稱其此等資力負擔賠償金,NaN,原告案發後仍與被告視訊\n原告證詞前後矛盾,2000000,80000
12,臺灣桃園地方法院106年訴字第572號民事判決,臺灣桃園地方法院民事判決106年度訴字第572號原告A1（即AA2（即AA3（即AA4（即A...,強制性交,"機車行,員工",其他專業﹑科學及技術服務業,員工,國中肄業,NaN,NaN,80000,2140000,NaN,因酒醉不知不能抗拒,NaN,並趁機性交,足致原告身心受到極大創傷,NaN,800000,500000


# Preprocessing

In [3]:
def education(text):
    if text!= None and type(text) == str:
        try:
            text = text.replace('\n','')
            seg_list = jieba.cut(text)
            seg_list = [s for s in seg_list]

            seg_list_1 = "".join(seg_list[:-1])
            seg_list_2 = seg_list[-1]

            if 'nan' in seg_list_2: return [0,0]
            if seg_list_1 == '': return [seg_list_2]
            return [seg_list_1,seg_list_2]
        except Exception as e:
            return [0,0]
    else: return [0,0]

def study(v):
    level = 0
    if type(v[0]) == str:
        text = v[0]
        if text in ['國小']: level = 0
        elif text in ['初中','國中','國 中']: level =  1
        elif text in ['高中','高職','高工','家商']: level =  2
        elif text in ['大學','空中大學','二專','專科','大專','技術學院']: level =  3
        elif text in ['研究','碩士']: level =  4
        elif text in ['博士']: level =  5
        if len(v)>1:
            if '肄業' in v[1]: return (level-0.5)
            else: return level

    
def extract_age(text):
    if type(text) == str:
        seg_list = jieba.cut(text)
        for s in seg_list:
            if s.isdigit(): return int(s)
    else:
        return text   
    

    
    

# 年齡處理
df['年齡'] = df['年齡'].fillna(0)
df['年齡'] = df['年齡'].apply(extract_age)
age_count = [v for v in df['年齡'] if v > 0]
df['年齡'] = df['年齡'].apply(lambda x : x if x>0 else np.mean(age_count)) # 年齡填充


# 學歷處理  
df['學歷'] = df['學歷'].apply(education)
level_dict = [v[0] for v in df['學歷'] if type(v[0]) == str]
level_dict = ['國小','初中','國中','國 中','高中','高職','高工','大學','空中大學','二專','碩士']           
df['學歷'] = df['學歷'].apply(study)  
study_count = [v for v in df['學歷'] if v > 0]
df['學歷'] = df['學歷'].apply(lambda x : x if x>0 else np.mean(study_count)) # 學歷填充

# 原告年紀處理 
df['原告年紀'] = df['原告年紀'].fillna(0)
hurt_age_count = [int(v) for v in df['原告年紀'] if int(v) > 0]
df['原告年紀'] = df['原告年紀'].apply(lambda x : x if x>0 else np.max(hurt_age_count)) # 原告年紀

# df
df[['職業','產業','階級','學歷','年齡']].tail()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\jack\AppData\Local\Temp\jieba.cache
Loading model cost 1.081 seconds.
Prefix dict has been built succesfully.


,職業,產業,階級,學歷,年齡
221,",公車司機",公共汽車客運業,員工,1.954955,36.272727
222,",法師",其他個人服務業,老闆,2.000000,36.272727
223,待業,NaN,NaN,1.954955,18.000000
224,",戲劇製作人","創作及藝術表演業, 藝文服務業",主管,1.954955,36.272727
225,待業,NaN,NaN,1.954955,36.272727


# 薪情平臺匯出資料_數額引用

In [4]:
# 數額填充
import operator
employee = pd.read_excel('Lawsnote法律黑客松＿判決摘要.xlsx',sheet_name='薪情平臺匯出資料') # 薪情平臺匯出資料

employee.head()  
employee_money ={} # 薪資排行榜
employee_mapping={} 
for idx, row in employee.iterrows():
    employee_money[list(row)[0]] = list(row)[2]    
    if type(list(row)[1]) == str:
        employee_mapping[list(row)[0]] = list(row)[1].split(',')
    

employee_rank = sorted(employee_money.items(), key=operator.itemgetter(1),reverse=True)
employee_rank = [e[0] for e in employee_rank]          

def fill_money(row):
    if row['數額']: return row['數額']
    else:
        return employee_money[row['產業']]

    
    
# df['數額'] = df.apply(fill_money)
df['階級'] = df['階級'].fillna('待業')
df['產業'] = df['產業'].fillna('待業')
# df['財產種類'] = df['財產種類'].fillna('無財產')
df[['職業','產業','階級','學歷','年齡','數額']].head()

,職業,產業,階級,學歷,年齡,數額
0,"餐飲及溫泉會館,老闆",住宿及餐飲業,老闆,1.954955,36.272727,1000000
2,待業,待業,待業,2.500000,36.272727,52540
3,待業,待業,待業,1.954955,36.272727,36000
4,醫師,醫療保健業,員工,1.954955,36.272727,1914876
5,待業,待業,待業,2.500000,36.272727,0


# 把犯罪類型、階級、財產種類做one hot encoding

In [5]:
# 刪除不必要的columns
df = df.drop(["法院", "主文", "職業", "產業", "等級", "財產總額"], axis=1)
# 所有\n化為，
df.replace('\n','，', inplace=True, regex=True)

# 刪除犯罪類型為nan的index
Nanlist = df[df["犯罪類型"].isnull().values==True].index.tolist()
df = df.drop(Nanlist)

df.reset_index(drop=True, inplace=True)
df.head()

,犯罪類型,階級,學歷,年齡,數額,財產種類,原告主張,原告年紀,被告主張,有利原告,有利被告,原告要求金額,最後判賠金額
0,強制性交,老闆,1.954955,36.272727,1000000,NaN,服用藥物昏沈不能抗拒,52,,趁原告服用安眠藥法抗拒，事後悔意從未道歉，指稱原告設局引誘被告,NaN,800000,500000
1,強制性交未遂,待業,2.500000,36.272727,52540,NaN,因酒醉不知不能抗拒,52,當日因飲酒過量精神恍惚，證人之證詞互有矛盾，DNA未為23種型別比對，請求之賠償金額過高,被告利用原告酒醉不知不能抗拒,NaN,1300000,900000
2,性騷擾,待業,1.954955,36.272727,36000,"投資:1,不動產:1",被告不滿原告以徒手拍打其臀部，而徒手反推原告，原告當場倒地，造成原告背部外傷及骨折,52,正當防衛，傷害故意，被告才是受害人,NaN,NaN,700000,50000
3,性騷擾,員工,1.954955,36.272727,1914876,NaN,被告趁原告尋求醫療協助之際，要求發生性行為，醫療行為不當，致原告腿部神經病變,52,術前已盡評估之事，原告為護士，於術前應已了解風險,趁病人不適且法逃脫醫師治療之依賴，違反道德,NaN,300000,250000
4,強制性交,待業,2.500000,36.272727,0,NaN,被告明知其為低智商而強制性交,52,二人互有情愫，並未暴力脅迫原告，經濟能力不佳，不應請求高額求償金,被告明知其為低智商而強制性交,NaN,1000000,400000


In [6]:
# df["財產種類"].unique()

In [7]:
#犯罪類型 one hot encoding
crime_df = pd.get_dummies(df["犯罪類型"])
crime_df.head()

,乘機性交,強制性交,強制性交、猥褻,強制性交未遂,性騷擾,猥褻,與未成年人合意性交
0,0,1,0,0,0,0,0
1,0,0,0,1,0,0,0
2,0,0,0,0,1,0,0
3,0,0,0,0,1,0,0
4,0,1,0,0,0,0,0


In [8]:
#階級 one hot encoding
level_df = pd.get_dummies(df["階級"])
level_df.head()

,主管,員工,待業,老闆
0,0,0,0,1
1,0,0,1,0
2,0,0,1,0
3,0,1,0,0
4,0,0,1,0


In [9]:
# 財產種類 one hot encoding
t_investment = 0   # 投資
t_real_estate = 0  # 不動產
t_car = 0          # 汽車
t_house = 0        # 房屋
t_land = 0         # 土地
t_motorcycle = 0   # 機車

investment = []   # 投資
real_estate = []  # 不動產
car = []          # 汽車
house = []        # 房屋
land = []         # 土地
motorcycle = []   # 機車

for index in df["財產種類"]:
    if not pd.isnull(index):
        comma = index.split(",")
        for item in comma:
            temp = item.split(":")
            if temp[1] == "n":        # n設定為5
                temp[1] = 5
            
            if temp[0] == "投資":
                t_investment = temp[1]
            elif temp[0] == "不動產":
                t_real_estate = temp[1]
            elif temp[0] == "汽車":
                t_car = temp[1]
            elif temp[0] == "房屋":
                t_house = temp[1]
            elif temp[0] == "土地":
                t_land = temp[1]
            elif temp[0] == "機車":
                t_motorcycle = temp[1]
        investment.append(t_investment)
        real_estate.append(t_real_estate)
        car.append(t_car)
        house.append(t_house)
        land.append(t_land)
        motorcycle.append(t_motorcycle)
        
        t_investment = 0   # 投資
        t_real_estate = 0  # 不動產
        t_car = 0          # 汽車
        t_house = 0        # 房屋
        t_land = 0         # 土地
        t_motorcycle = 0   # 機車
    else:
        investment.append(0)
        real_estate.append(0)
        car.append(0)
        house.append(0)
        land.append(0)
        motorcycle.append(0)

c = {"投資":investment,
     "不動產":real_estate,
     "汽車":car,
     "房屋":house,
     "土地":land,
     "機車":motorcycle}
belongings = pd.DataFrame(c)
belongings.head()

,投資,不動產,汽車,房屋,土地,機車
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,1,1,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


In [10]:
# 刪除columns(犯罪類型、階級、財產種類、原告年紀)
df = df.drop(["犯罪類型", "階級", "財產種類", "原告年紀"], axis=1)
df.head()

,學歷,年齡,數額,原告主張,被告主張,有利原告,有利被告,原告要求金額,最後判賠金額
0,1.954955,36.272727,1000000,服用藥物昏沈不能抗拒,,趁原告服用安眠藥法抗拒，事後悔意從未道歉，指稱原告設局引誘被告,NaN,800000,500000
1,2.500000,36.272727,52540,因酒醉不知不能抗拒,當日因飲酒過量精神恍惚，證人之證詞互有矛盾，DNA未為23種型別比對，請求之賠償金額過高,被告利用原告酒醉不知不能抗拒,NaN,1300000,900000
2,1.954955,36.272727,36000,被告不滿原告以徒手拍打其臀部，而徒手反推原告，原告當場倒地，造成原告背部外傷及骨折,正當防衛，傷害故意，被告才是受害人,NaN,NaN,700000,50000
3,1.954955,36.272727,1914876,被告趁原告尋求醫療協助之際，要求發生性行為，醫療行為不當，致原告腿部神經病變,術前已盡評估之事，原告為護士，於術前應已了解風險,趁病人不適且法逃脫醫師治療之依賴，違反道德,NaN,300000,250000
4,2.500000,36.272727,0,被告明知其為低智商而強制性交,二人互有情愫，並未暴力脅迫原告，經濟能力不佳，不應請求高額求償金,被告明知其為低智商而強制性交,NaN,1000000,400000


# Word2Vec

In [11]:
#停用詞
stopwords=[]
for word in open('stopwords.txt','r', encoding='utf8'):
    stopwords.append(word.strip())

In [12]:
print(sys.getdefaultencoding())

utf-8


# 原告主張

In [13]:
# 原告主張word2vec
df.原告主張 = df.原告主張.astype(str)
title_file = open("title_split.txt", "w", encoding = 'utf8')

#分詞(title和keyword)
splitTitle = []
for i in range(len(df.原告主張)):
    split = jieba.cut(df.原告主張[i], cut_all=False)
    temp = ""
    for word in split:
        if word not in stopwords:  #若word不在停用詞裡面，則加入
            temp = temp + word + " "
    temp = temp.strip()  #刪除不必要的空白
    
    title_file.writelines(str(temp + "\n"))   #存檔(分割詞，已扣除stopword)
    splitTitle.append(temp)

title_file.close()

In [14]:
sentences = word2vec.LineSentence("title_split.txt")
model = word2vec.Word2Vec(sentences, size=300, min_count=1)
print(model)

Word2Vec(vocab=1951, size=300, alpha=0.025)


In [15]:
# 这里主要是要求得能代表文档的向量，这里就简单的将文档中的句子相加求平均，得到一个300维的文档向量
vecs = []
    
for words in sentences:
    # vec = np.zeros(2).reshape((1, 2))
    vec = np.zeros(300).reshape((1, 300))
    count = 0
    # words = remove_some(words)
    for word in words[1:]:
        try:
            count = count + 1
            # vec += model[word].reshape((1, 2))
            vec = vec + model[word].reshape((1, 300))
#             print(vec)
        except KeyError:
            continue
#     if count > 0:
    vec = vec/count
    vecs.append(vec)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide


In [16]:
#要先轉乘np.array
vecs = np.array(vecs, dtype='float64')
print(vecs.shape)

(210, 1, 300)


In [17]:
# 3維轉2維
vecs = np.reshape(vecs, (len(vecs),300))
print(vecs.shape)

(210, 300)


In [18]:
vector0 = pd.DataFrame(vecs)

# 被告主張

In [19]:
# 被告主張word2vec
df["被告主張"] = df["被告主張"].replace('','無')
df.被告主張 = df.被告主張.astype(str)
title_file = open("title_split.txt", "w", encoding = 'utf8')

#分詞(title和keyword)
splitTitle = []
for i in range(len(df.被告主張)):
    split = jieba.cut(df.被告主張[i], cut_all=False)
    temp = ""
    for word in split:
#         if word not in stopwords:  #若word不在停用詞裡面，則加入
        temp = temp + word + " "
    temp = temp.strip()  #刪除不必要的空白
    
    title_file.writelines(str(temp + "\n"))   #存檔(分割詞，已扣除stopword)
    splitTitle.append(temp)

title_file.close()

In [20]:
sentences = word2vec.LineSentence("title_split.txt")
model = word2vec.Word2Vec(sentences, size=300, min_count=1)
print(model)

Word2Vec(vocab=1276, size=300, alpha=0.025)


In [21]:
# 这里主要是要求得能代表文档的向量，这里就简单的将文档中的句子相加求平均，得到一个300维的文档向量
vecs = []
    
for words in sentences:
    # vec = np.zeros(2).reshape((1, 2))
    vec = np.zeros(300).reshape((1, 300))
    count = 0
    # words = remove_some(words)
    for word in words[1:]:
        try:
            count = count + 1
            # vec += model[word].reshape((1, 2))
            vec = vec + model[word].reshape((1, 300))
#             print(vec)
        except KeyError:
            continue
#     if count > 0:
    vec = vec/count
    vecs.append(vec)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [22]:
#要先轉乘np.array
vecs = np.array(vecs, dtype='float64')
print(vecs.shape)

# 3維轉2維
vecs = np.reshape(vecs, (len(vecs),300))
print(vecs.shape)

vector1 = pd.DataFrame(vecs)

(210, 1, 300)
(210, 300)


# 有利原告

In [23]:
# 有利原告word2vec
df["有利原告"] = df["有利原告"].replace('','無')
df.有利原告 = df.有利原告.astype(str)
title_file = open("title_split.txt", "w", encoding = 'utf8')

#分詞(title和keyword)
splitTitle = []
for i in range(len(df.有利原告)):
    split = jieba.cut(df.有利原告[i], cut_all=False)
    temp = ""
    for word in split:
#         if word not in stopwords:  #若word不在停用詞裡面，則加入
        temp = temp + word + " "
    temp = temp.strip()  #刪除不必要的空白
    
    title_file.writelines(str(temp + "\n"))   #存檔(分割詞，已扣除stopword)
    splitTitle.append(temp)

title_file.close()

sentences = word2vec.LineSentence("title_split.txt")
model = word2vec.Word2Vec(sentences, size=300, min_count=1)
print(model)

# 这里主要是要求得能代表文档的向量，这里就简单的将文档中的句子相加求平均，得到一个300维的文档向量
vecs = []
    
for words in sentences:
    # vec = np.zeros(2).reshape((1, 2))
    vec = np.zeros(300).reshape((1, 300))
    count = 0
    # words = remove_some(words)
    for word in words[1:]:
        try:
            count = count + 1
            # vec += model[word].reshape((1, 2))
            vec = vec + model[word].reshape((1, 300))
#             print(vec)
        except KeyError:
            continue
#     if count > 0:
    vec = vec/count
    vecs.append(vec)
    
#要先轉乘np.array
vecs = np.array(vecs, dtype='float64')
print(vecs.shape)

# 3維轉2維
vecs = np.reshape(vecs, (len(vecs),300))
print(vecs.shape)

vector2 = pd.DataFrame(vecs)

Word2Vec(vocab=1770, size=300, alpha=0.025)
(210, 1, 300)
(210, 300)


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:37: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in true_divide


# 有利被告

In [24]:
# 有利被告word2vec
df["有利被告"] = df["有利被告"].replace('','無')
df.有利被告 = df.有利被告.astype(str)
title_file = open("title_split.txt", "w", encoding = 'utf8')

#分詞(title和keyword)
splitTitle = []
for i in range(len(df.有利被告)):
    split = jieba.cut(df.有利被告[i], cut_all=False)
    temp = ""
    for word in split:
#         if word not in stopwords:  #若word不在停用詞裡面，則加入
        temp = temp + word + " "
    temp = temp.strip()  #刪除不必要的空白
    
    title_file.writelines(str(temp + "\n"))   #存檔(分割詞，已扣除stopword)
    splitTitle.append(temp)

title_file.close()

sentences = word2vec.LineSentence("title_split.txt")
model = word2vec.Word2Vec(sentences, size=300, min_count=1)
print(model)

# 这里主要是要求得能代表文档的向量，这里就简单的将文档中的句子相加求平均，得到一个300维的文档向量
vecs = []
    
for words in sentences:
    # vec = np.zeros(2).reshape((1, 2))
    vec = np.zeros(300).reshape((1, 300))
    count = 0
    # words = remove_some(words)
    for word in words[1:]:
        try:
            count = count + 1
            # vec += model[word].reshape((1, 2))
            vec = vec + model[word].reshape((1, 300))
#             print(vec)
        except KeyError:
            continue
#     if count > 0:
    vec = vec/count
    vecs.append(vec)
    
#要先轉乘np.array
vecs = np.array(vecs, dtype='float64')
print(vecs.shape)

# 3維轉2維
vecs = np.reshape(vecs, (len(vecs),300))
print(vecs.shape)

vector3 = pd.DataFrame(vecs)

Word2Vec(vocab=665, size=300, alpha=0.025)
(210, 1, 300)
(210, 300)


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in true_divide
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:37: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [25]:
vector0 = vector0.fillna(0)
vector1 = vector1.fillna(0)
vector2 = vector2.fillna(0)
vector3 = vector3.fillna(0)

# 刪除不必要的columns，concat所有dataframe，並分割資料

In [26]:
label = df["最後判賠金額"]
# 刪除columns(原告主張, 被告主張, 有利原告, 有利被告, 原告要求金額, 最後判賠金額)
df = df.drop(["原告主張", "被告主張", "有利原告", "有利被告", "原告要求金額", "最後判賠金額"], axis=1)

In [27]:
final_df = pd.concat([df,crime_df,level_df,belongings,vector0,vector1,vector2,vector3],axis=1)

In [28]:
final_df.head()

,學歷,年齡,數額,乘機性交,強制性交,強制性交、猥褻,強制性交未遂,性騷擾,猥褻,與未成年人合意性交,...,290,291,292,293,294,295,296,297,298,299
0,1.954955,36.272727,1000000,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.500000,36.272727,52540,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.954955,36.272727,36000,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.954955,36.272727,1914876,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.500000,36.272727,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
final_data = np.array(final_df)
label = np.array(label, dtype='float64')

In [30]:
from sklearn.model_selection import train_test_split

train_X,test_X,train_y,test_y = train_test_split(final_data,label, test_size=0.10, random_state=0)

# 模型訓練(svm)

In [31]:
from sklearn.svm import SVR

# # 使用线性核函数配置的支持向量机进行回归训练，并且对测试样本进行预测。
# linear_svr = SVR(kernel='linear')
# linear_svr.fit(train_X, train_y)
# linear_svr_y_predict = linear_svr.predict(test_X)
# linear_svr_y_predict

# # 使用多项式核函数配置的支持向量机进行回归训练，并且对测试样本进行预测。
# poly_svr = SVR(kernel='poly')
# poly_svr.fit(train_X, train_y)
# poly_svr_y_predict = poly_svr.predict(test_X)

# # 使用径向基核函数配置的支持向量机进行回归训练，并且对测试样本进行预测。
# rbf_svr = SVR(kernel='rbf')
# rbf_svr.fit(train_X, train_y)
# rbf_svr_y_predict = rbf_svr.predict(test_X)

In [32]:
# C是懲罰參數，即是對誤差的寬容程度，c越高，说明越不能容忍出现误差,容易过拟合。C越小，容易欠拟合。C过大或过小，泛化能力变差
# gamma是选择RBF函数作为kernel后，该函数自带的一个参数。隐含地决定了数据映射到新的特征空间后的分布
# gamma越大，支持向量越少，gamma值越小，支持向量越多，支持向量的个数影响训练与预测的速度。
clf = SVR(gamma = 1e-9, C = 1e6)   
clf.fit(train_X, train_y) 
Y_pred = clf.predict(test_X)
Y_pred

array([ 349999.90076323,  642229.04261027,  642229.04432928,
        400000.07232024,  577466.56729409,  385435.56966234,
        324684.69947229,  400000.04136215,  694493.6775227 ,
        703537.43201926,  400000.06314729, 1630944.21426092,
        194988.78007766,  525033.43235497,  400000.05784824,
        349999.4587132 ,  896665.23128255,  349999.96226306,
        787248.60964407,  400000.09444649,  599869.95574376])

In [33]:
test_y

array([ 900000.,  400000.,  600000.,  300000.,  250000.,  150000.,
       1200000.,  300000.,  800000., 1500000., 1000000.,  400000.,
        800000.,  250000.,  800000.,  700000.,  500000., 1000000.,
       1200000.,  500000.,  400000.])

In [34]:
error = []
for i in range(len(test_y)):
    error.append(test_y[i] - Y_pred[i])
    
#print("Errors: ", error)
#print(error)
squaredError = []
absError = []
for val in error:
    squaredError.append(val * val)#平方
    absError.append(abs(val))#誤差絕對值
    
#print("Square Error: ", squaredError)
#print("Absolute Value of Error: ", absError)
print("MSE = ", sum(squaredError) / len(squaredError))#平均平方誤差MSE


#from math import sqrt
#print("RMSE = ", sqrt(sum(squaredError) / len(squaredError)))#平均平方根誤差RMSE
print("MAE = ", sum(absError) / len(absError))#平均絕對誤差MAE

MSE =  255087838137.06897
MAE =  409281.92549062346
